<a href="https://colab.research.google.com/github/Bahaakhalled/Car-Price-Prediction/blob/main/CarPricesFull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **`Car Prediction using Linear regression`**

---



---



## Importing Libraries


In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from sklearn.impute import SimpleImputer


In [ ]:
#data1=files.upload()

###Inserting data into Notebook

In [ ]:
cars=pd.read_csv('Car details v3.csv')

In [ ]:
cars.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


##Getting to know the data

In [ ]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [ ]:
cars.shape

(8128, 13)

###Missing Data Percentage

In [ ]:
total_cells=np.product(cars.shape)
total_missing=cars.isnull().sum().sum()
percent_missing=int((total_missing / total_cells)*100)
print(str(percent_missing) + '% of the data is missing')

1% of the data is missing


Had only one Value as Electric under the fuel Column so removed it as not enough data to represent the Electric category in future Data

In [ ]:
cars=cars.drop(cars.loc[cars.fuel=='Electric'].index)

In [ ]:
cars.fuel.value_counts()

Diesel    4402
Petrol    3631
CNG         57
LPG         38
Name: fuel, dtype: int64

Created a new Column named Brand with Only the brand of the car without extra details in car name then dropped the name column

In [ ]:
cars['Brand']=cars.name.map(lambda l:(l.split()[0]))

In [ ]:
cars['Brand']

0        Maruti
1         Skoda
2         Honda
3       Hyundai
4        Maruti
         ...   
8123    Hyundai
8124    Hyundai
8125     Maruti
8126       Tata
8127       Tata
Name: Brand, Length: 8128, dtype: object

In [ ]:
cars=cars.drop("name",axis=1)

In [ ]:
cars

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,Brand
0,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,Maruti
1,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda
2,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Honda
3,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0,Hyundai
4,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Maruti
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197 CC,82.85 bhp,113.7Nm@ 4000rpm,5.0,Hyundai
8124,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493 CC,110 bhp,"24@ 1,900-2,750(kgm@ rpm)",5.0,Hyundai
8125,2009,382000,120000,Diesel,Individual,Manual,First Owner,19.3 kmpl,1248 CC,73.9 bhp,190Nm@ 2000rpm,5.0,Maruti
8126,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396 CC,70 bhp,140Nm@ 1800-3000rpm,5.0,Tata


Filled Missing Seats Columns with the Columns Mode

In [ ]:
cars['seats']=cars.seats.fillna(cars['seats'].mode()[0])

In [ ]:
cars.seats.value_counts()

5.0     6475
7.0     1120
8.0      236
4.0      133
9.0       80
6.0       62
10.0      19
2.0        2
14.0       1
Name: seats, dtype: int64

In [ ]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           8128 non-null   int64  
 1   selling_price  8128 non-null   int64  
 2   km_driven      8128 non-null   int64  
 3   fuel           8128 non-null   object 
 4   seller_type    8128 non-null   object 
 5   transmission   8128 non-null   object 
 6   owner          8128 non-null   object 
 7   mileage        7907 non-null   object 
 8   engine         7907 non-null   object 
 9   max_power      7913 non-null   object 
 10  torque         7906 non-null   object 
 11  seats          8128 non-null   float64
 12  Brand          8128 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 889.0+ KB


In [ ]:
#cars.loc[cars.max_power.isin(['/'])]

In [ ]:
cars.transmission.unique()

array(['Manual', 'Automatic'], dtype=object)

Function Below to remove String words in Object Type Column followed by transforming each Column to a numerical form for Regression participation

In [ ]:
def removeextra(columnname,toremove):
  cars[columnname]=cars[columnname].str.rstrip(toremove)

In [ ]:
removeextra('max_power',' bhp')

In [ ]:
cars['max_power'][1]

'103.52'

In [ ]:
cars['max_power']=cars.max_power.replace('','nan')

In [ ]:
cars['max_power']=cars['max_power'].astype('float',errors='raise')

In [ ]:
cars.max_power.dtype

dtype('float64')

In [ ]:
removeextra('engine',' CC')
cars['engine']=cars['engine'].astype('float')

Had two different Scales in Mileage but googled the difference in scale and it turns out to be EU and US of the same measurement

In [ ]:
removeextra('mileage',' kmpl')
removeextra('mileage',' km/kg')
cars['mileage']=cars['mileage'].astype('float')

In [ ]:
cars['mileage'].map(lambda m:'km/kg' in str(m)).sum()

0

In [ ]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           8128 non-null   int64  
 1   selling_price  8128 non-null   int64  
 2   km_driven      8128 non-null   int64  
 3   fuel           8128 non-null   object 
 4   seller_type    8128 non-null   object 
 5   transmission   8128 non-null   object 
 6   owner          8128 non-null   object 
 7   mileage        7907 non-null   float64
 8   engine         7907 non-null   float64
 9   max_power      7912 non-null   float64
 10  torque         7906 non-null   object 
 11  seats          8128 non-null   float64
 12  Brand          8128 non-null   object 
dtypes: float64(4), int64(3), object(6)
memory usage: 1.1+ MB


Dropped Missing Rows as they only represent 1% of the data

In [ ]:
nomissingcars=cars.dropna( axis=0, how="any", thresh=None, subset=None, inplace=False)

In [ ]:
nomissingcars.shape

(7906, 13)

In [ ]:
nomissingcars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7906 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           7906 non-null   int64  
 1   selling_price  7906 non-null   int64  
 2   km_driven      7906 non-null   int64  
 3   fuel           7906 non-null   object 
 4   seller_type    7906 non-null   object 
 5   transmission   7906 non-null   object 
 6   owner          7906 non-null   object 
 7   mileage        7906 non-null   float64
 8   engine         7906 non-null   float64
 9   max_power      7906 non-null   float64
 10  torque         7906 non-null   object 
 11  seats          7906 non-null   float64
 12  Brand          7906 non-null   object 
dtypes: float64(4), int64(3), object(6)
memory usage: 864.7+ KB


In [ ]:
 # Scale After switching to Numericals

In [ ]:
X=nomissingcars

In [ ]:
X['Brand'].nunique()

31

Created X and y to start the Prediction Stages by removing the Torque along with the Brand Column for now aswell as the predictive selling price

In [ ]:
X=X.drop(["selling_price","torque","Brand"],axis=1)

In [ ]:
y=nomissingcars['selling_price']
y.shape

(7906,)

In [ ]:
X

,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
0,2014,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,5.0
1,2014,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,2006,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,5.0
3,2010,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,5.0
4,2007,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,5.0
...,...,...,...,...,...,...,...,...,...,...
8123,2013,110000,Petrol,Individual,Manual,First Owner,18.50,1197.0,82.85,5.0
8124,2007,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.80,1493.0,110.00,5.0
8125,2009,120000,Diesel,Individual,Manual,First Owner,19.30,1248.0,73.90,5.0
8126,2013,25000,Diesel,Individual,Manual,First Owner,23.57,1396.0,70.00,5.0


In [ ]:
train_X,val_X,train_y,val_y=train_test_split(X,y,random_state=0)

# Converting Categorical Columns to Numerical after train and test split


Splitted numerical and categorical columns to start Functioning on each seperately

In [ ]:
numerical_cols=X.select_dtypes(exclude="object").columns
numerical_cols

Index(['year', 'km_driven', 'mileage', 'engine', 'max_power', 'seats'], dtype='object')

In [ ]:
cat_cols=X.select_dtypes(include="object").columns
cat_cols

Index(['fuel', 'seller_type', 'transmission', 'owner'], dtype='object')

In [ ]:
temp_train_X=train_X
temp_val_X=val_X
temp_train_X

,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
1075,2016,94000,Diesel,Dealer,Automatic,First Owner,11.36,2755.0,171.50,7.0
3139,2015,90000,Diesel,Individual,Manual,Second Owner,25.80,1498.0,98.60,5.0
1682,2015,20000,Diesel,Individual,Manual,First Owner,20.40,1248.0,91.70,5.0
5555,2018,50000,Petrol,Individual,Manual,First Owner,21.63,998.0,67.00,5.0
6571,2016,11000,Petrol,Dealer,Automatic,First Owner,14.30,1598.0,103.50,5.0
...,...,...,...,...,...,...,...,...,...,...
5074,2011,105000,Petrol,Individual,Manual,First Owner,17.00,1497.0,118.00,5.0
3367,2016,38000,Petrol,Individual,Manual,First Owner,18.60,1197.0,81.83,5.0
1703,2015,35000,Petrol,Individual,Manual,First Owner,17.30,1497.0,117.30,7.0
2694,2013,173000,Diesel,Individual,Manual,First Owner,12.05,2179.0,120.00,7.0


Used get_dummies to One Hot Encode Categorical Columns

In [ ]:
cat_train=pd.get_dummies(temp_train_X)
cat_val=pd.get_dummies(temp_val_X)
cat_train.columns

Index(['year', 'km_driven', 'mileage', 'engine', 'max_power', 'seats',
       'fuel_CNG', 'fuel_Diesel', 'fuel_LPG', 'fuel_Petrol',
       'seller_type_Dealer', 'seller_type_Individual',
       'seller_type_Trustmark Dealer', 'transmission_Automatic',
       'transmission_Manual', 'owner_First Owner',
       'owner_Fourth & Above Owner', 'owner_Second Owner',
       'owner_Test Drive Car', 'owner_Third Owner'],
      dtype='object')

Tried Manipulating with Features to form a better result with lesser Error

In [ ]:
cat_train=cat_train.drop(['seats','owner_Fourth & Above Owner'],axis=1)
cat_val=cat_val.drop(['seats','owner_Fourth & Above Owner'],axis=1)

In [ ]:
model=LinearRegression()

Below Function Fits,Predicts and returns MSE

In [ ]:
def Modeltoerror():
  #my_pipeline=Pipeline(steps=[('preprocessor',preprocessor),('model',model)])
  model.fit(cat_train,train_y)
  preds=model.predict(cat_val)
  return mean_absolute_error(val_y,preds)

Least Error With Current Features and Progress (6/29/2022)

In [ ]:
Modeltoerror()

279823.2977207504

Decision Tree Regressor with no Parameter Modification provided much better Results than Linear Regression

In [ ]:
model=DecisionTreeRegressor()

In [ ]:
Modeltoerror()

80681.98474119035